In [5]:
# Dependencies and Setup

import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/Output/city_weather.csv")

# Display sample data
city_data_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,1490140,Tara,RU,56.8944,74.3710,62.35,96,86,6.20
1,1546102,Port-aux-Français,TF,-49.3500,70.2167,35.56,60,64,37.47
2,2454934,Kolondiéba,ML,11.0882,-6.8926,85.89,59,83,8.50
3,4513583,Hamilton,US,39.1834,-84.5333,86.18,68,75,9.22
4,4030723,Adamstown,PN,-25.0660,-130.1015,73.24,80,99,36.13
...,...,...,...,...,...,...,...,...,...
563,2315026,Kasongo-Lunda,CD,-6.4667,16.8167,80.71,32,46,2.24
564,5847504,Kailua-Kona,US,19.6406,-155.9956,80.40,86,75,4.61
565,601299,Muynak,UZ,43.7683,59.0214,82.65,33,5,5.06
566,1159301,Zahedan,IR,29.4963,60.8629,87.71,20,100,9.22


In [ ]:
# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. 
# The size of the point should be the humidity in each city.

In [7]:
# %%capture --no-display

# Configure the map plot
map_plot_1 =city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size= "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width =700,
    frame_height =500,
    scale = 1,
    color = "City"
)
# YOUR CODE HERE

# Display the map

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [11]:
# Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

# Narrow down cities that fit criteria and drop any results with null values

temp_df = city_data_df[(city_data_df["Temperature"] >= 75) & (city_data_df["Temperature"] <= 90)]
wind_df = temp_df[temp_df["Wind Speed"] <= 10]
weather_df = wind_df[wind_df["Clouds"] <=10]
weather_df = wind_df[wind_df["Humidity"] <=70]
weather_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
2,2454934,Kolondiéba,ML,11.0882,-6.8926,85.89,59,83,8.50
3,4513583,Hamilton,US,39.1834,-84.5333,86.18,68,75,9.22
5,5106841,Albany,US,42.6001,-73.9662,82.53,48,3,1.99
27,5282297,Bethel,US,41.3712,-73.4140,82.65,59,40,5.75
48,6185377,Yellowknife,CA,62.4560,-114.3525,79.32,44,100,5.75
...,...,...,...,...,...,...,...,...,...
546,2214846,Misratah,LY,32.3754,15.0925,85.69,64,0,9.04
563,2315026,Kasongo-Lunda,CD,-6.4667,16.8167,80.71,32,46,2.24
565,601299,Muynak,UZ,43.7683,59.0214,82.65,33,5,5.06
566,1159301,Zahedan,IR,29.4963,60.8629,87.71,20,100,9.22


In [21]:
# Step 3: Create a new DataFrame called hotel_df.

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = weather_df.drop(columns = ["City ID", "Temperature", "Clouds", "Wind Speed"]).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,Kolondiéba,ML,11.0882,-6.8926,59,
3,Hamilton,US,39.1834,-84.5333,68,
5,Albany,US,42.6001,-73.9662,48,
27,Bethel,US,41.3712,-73.4140,59,
48,Yellowknife,CA,62.4560,-114.3525,44,
...,...,...,...,...,...,...
546,Misratah,LY,32.3754,15.0925,64,
563,Kasongo-Lunda,CD,-6.4667,16.8167,32,
565,Muynak,UZ,43.7683,59.0214,33,
566,Zahedan,IR,29.4963,60.8629,20,


In [22]:
# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
limit= 1
params = {
    "categories":categories,
    "limit":limit,   
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{'Lon'},{'Lat'},{radius}"
    params["bias"] = f"proximity:{'Lon'},{'Lat'}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
Kolondiéba - nearest hotel: No hotel found
Hamilton - nearest hotel: No hotel found
Albany - nearest hotel: No hotel found
Bethel - nearest hotel: No hotel found
Yellowknife - nearest hotel: No hotel found
Guajará-Mirim - nearest hotel: No hotel found
Catherinas Lust - nearest hotel: No hotel found
Kentville - nearest hotel: No hotel found
Jackson - nearest hotel: No hotel found
Kongolo - nearest hotel: No hotel found
Cambria - nearest hotel: No hotel found
Laguna - nearest hotel: No hotel found
Chu - nearest hotel: No hotel found
Danville - nearest hotel: No hotel found
Patterson - nearest hotel: No hotel found
Thompson - nearest hotel: No hotel found
Teays Valley - nearest hotel: No hotel found
Zhangatas - nearest hotel: No hotel found
Selma - nearest hotel: No hotel found
North Gates - nearest hotel: No hotel found
Cuerámaro - nearest hotel: No hotel found
Mashhad - nearest hotel: No hotel found
Clarion - nearest hotel: No hotel found
Bay City - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
2,Kolondiéba,ML,11.0882,-6.8926,59,No hotel found
3,Hamilton,US,39.1834,-84.5333,68,No hotel found
5,Albany,US,42.6001,-73.9662,48,No hotel found
27,Bethel,US,41.3712,-73.4140,59,No hotel found
48,Yellowknife,CA,62.4560,-114.3525,44,No hotel found
...,...,...,...,...,...,...
546,Misratah,LY,32.3754,15.0925,64,No hotel found
563,Kasongo-Lunda,CD,-6.4667,16.8167,32,No hotel found
565,Muynak,UZ,43.7683,59.0214,33,No hotel found
566,Zahedan,IR,29.4963,60.8629,20,No hotel found


In [23]:
# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = "all"
)
# Display the map

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)